In [17]:
num_epochs = 10
batch_size = 32
best_accuracy = 0

In [18]:
import torch
import torch.nn as nn
import numpy as np
import os
import pandas as pd
import dask.dataframe as dd
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
import torch.nn.functional as F
from IPython.display import clear_output
import time

In [19]:
class MyCNN(nn.Module):
    def __init__(self, num_layers = 10, input_features=153, output_features=1):
        super(MyCNN, self).__init__()
        
        self.conv_layers = nn.ModuleList()
        self.conv_layers.append(nn.Conv1d(in_channels=1, out_channels=256, kernel_size=3, padding=1))
        
        for _ in range(1, num_layers - 1):
            self.conv_layers.append(nn.Conv1d(in_channels=256, out_channels=256, kernel_size=3, padding=1))
        
        self.conv_layers.append(nn.Conv1d(in_channels=256, out_channels=1, kernel_size=1, padding=0))
        
        self.activation = nn.LeakyReLU(negative_slope=0.01)

        self.fc = nn.Linear(input_features, output_features)
    def forward(self, x):
        #x = x.unsqueeze(-1)

        for i, conv in enumerate(self.conv_layers):
            if(i < len(self.conv_layers) - 1):
                x = self.activation(conv(x))
            else:
                x = conv(x)
        x = x.squeeze(1)
        x = self.fc(x)
        x = torch.sigmoid(x)
        print("x.shape:",x.shape)
        
        return x

In [20]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

Using device: cuda


In [21]:
model = torch.load('E:/專題/chow_model/CNN_chow_model_50.0.pth')
model.to(device)
loss_criterion  = nn.BCELoss()
model.eval()

C:\Users\timy3\AppData\Local\Temp\ipykernel_108860\2763338689.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load('E:/專題/chow_model/CNN_chow_model_50.0.pt

MyCNN(
  (conv_layers): ModuleList(
    (0): Conv1d(1, 256, kernel_size=(3,), stride=(1,), padding=(1,))
    (1-8): 8 x Conv1d(256, 256, kernel_size=(3,), stride=(1,), padding=(1,))
    (9): Conv1d(256, 1, kernel_size=(1,), stride=(1,))
  )
  (activation): LeakyReLU(negative_slope=0.01)
  (fc): Linear(in_features=153, out_features=1, bias=True)
)

In [22]:
class MyDataset(Dataset):
    def __init__(self, file_path, chunk_size):
        self.file_path = file_path
        self.chunk_size = chunk_size
        
        self.columns = pd.read_csv(self.file_path, nrows=0).columns.str.strip().str.replace('\xa0', ' ').tolist()
        self.train_columns = [col for col in self.columns if col != 'If Chow']  # 排除 'If Chow' 欄位
        self.num_samples = sum(1 for _ in open(self.file_path, encoding='utf-8')) - 1  # 計算總樣本數
        
    def __getitem__(self, idx):
        chunk_start = idx // self.chunk_size * self.chunk_size
        df = pd.read_csv(self.file_path, skiprows=chunk_start + 1, nrows=self.chunk_size, header=None, encoding='utf-8')  # 跳過標題和之前的行

        df.columns = self.columns
        
        if 'If Chow' not in df.columns:
            raise KeyError(f"Chunk starting at row {chunk_start + 1} does not contain 'If Chow' column.")
        
            
        sample_idx = idx % self.chunk_size
        
        train_data = df[self.train_columns].iloc[sample_idx].values
        
        value_data = df['If Chow'].iloc[sample_idx]  # 提取 'If Chow' 欄位
        
        return torch.tensor(train_data, dtype=torch.float32), torch.tensor(value_data, dtype=torch.float32)
        
    def __len__(self):
        return self.num_samples

In [23]:
"""
讀取csv
"""
file_path = 'E:/專題/data/2022/ChowData.csv'
dataset = MyDataset(file_path, chunk_size=50000)  # 每次只加載 50000 行
test_dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False)

In [ ]:
correct = 0
total = 0
with torch.no_grad():
    for train_data, value_data in test_dataloader:
        torch.autograd.set_detect_anomaly(True)

        channel1_tensor = train_data.unsqueeze(1).to(device)
        value_data = value_data.to(device).float()
        
        output = model(channel1_tensor).squeeze(1)

        loss = loss_criterion(output, value_data)
        current_loss = loss.item()

        clear_output()
        predicted_labels = (output > 0.5).float()
        correct_predictions = (predicted_labels == value_data).sum().item()
        correct += correct_predictions
        total += value_data.size(0)
        accuracy = (correct_predictions / value_data.size(0)) * 100
        totalAccuracy = (correct/total)*100
        print(predicted_labels)
        print(value_data)
        print(f"Correct Predictions: {correct_predictions}")
        print(f"Total Predictions: {value_data.size(0)}")
        print(f"Cur Accuracy: {accuracy:.4f}","%")
        print(f"Total Accuracy: {totalAccuracy:.4f}","%")
        print(f"loss: {current_loss}")
        print(f"best accuracy: {best_accuracy}")

tensor([1., 1., 1., 1., 1., 1., 0., 0., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0.],
       device='cuda:0')
tensor([0., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0.,
        1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0.],
       device='cuda:0')
Correct Predictions: 14
Total Predictions: 32
Cur Accuracy: 43.7500 %
Total Accuracy: 51.2074 %
loss: 0.7189125418663025
best accuracy: 0
